# Lecture 01. USB Camera 를 활용하여 데이터 수집하기

### Contents:
   
   1. Lecture 01 개요
   
   2. USB camera 스펙
   
   3. USB camera 연결 및 동작 확인
   
   4. Smart Factory의 컨베이어벨트를 비디오로 파일로 저장하기
   
   5. 비디오 파일이 잘 생성됐는지 확인하기
   
   ---
   

## 0. 사전 작업
Jupyter Notebook 에서 필요한 함수를 define 하자.

In [17]:
# Import the libraries needed to use Jupyter Notebook
import ipywidgets as widgets
import IPython.display as display
from IPython.display import clear_output

import cv2

widget_dict = {}

def init_display():
    clear_output()
    widget_dict.clear()

# Can be replaced with cv2.imshow() function when used in terminal
def imshow(title, img):
    if title not in widget_dict:
        print(title)

        # Set the widget to display the screen in the Jupyter Notebook environment
        widget_dict[title] = widgets.Image(
            layout = widgets.Layout(border="solid")
        )
        display.display(widget_dict[title])
  
    jpg_stream = cv2.imencode(".jpg", img)[1].tostring()
    widget_dict[title].value = jpg_stream
    
def show_video(title, file):
    init_display()
    cap = cv2.VideoCapture(file)
    while True:
        res, img = cap.read()
        if res is False:
            break
        imshow(title, img)
    cap.release()

---
## **1. Lecture 01 개요**


이번 강의에서는 USB Camera 를 활용하여 Camera Streaming Input 을 비디오 파일로 저장한다. 이 파일은 다음 강의에서 지속적으로 사용된다.  <p>

이번 강의에서는:<p>
- USB camera input 을 OpenCV 를 활용하여 비디오 파일로 저장하는 법을 배운다.

In [3]:
import cv2
cap = cv2.VideoCapture(0)

---
## **2. USB camera 스펙 (D455)**

 - RGB Frame Rate: 640x480@30fps, 

In [4]:
FPS = cap.get(cv2.CAP_PROP_FPS)

In [5]:
W = int(cap.get(3))
H = int(cap.get(4))

In [6]:
print('FPS: %d, width: %d, height: %d' % (FPS,W, H))

FPS: 30, width: 640, height: 480


---
## **3. USB camera 연결 및 동작 확인**

### 3.1. Camera support for Windows

<br>
Linux의 경우 V4L2 (Video For Linux 2) 는 Linux 에서 camera 를 사용 할 수 있게 해주는 interface 이다. 이 interface 를 통해 camera device 를 제어하고 application 과 통신 할 수 있지만 Windows의 경우 카메라 Interface를 사용할 수 있는 Library를 찾기 힘들기 때문에 보통 openCV의 VideosCapture 함수를 통해 숫자로서 장치를 확인 할 수 있다.<br>

In [ ]:
while True:
    try:
        show_video('', 1)#trying 0 or 1 device
    except KeyboardInterrupt:  # input the 'ii' key
        break

---
## **4. Smart Factory의 컨베이어벨트를 비디오로 파일로 저장하기**

아래와 같은 형태의 비디오 파일을 각자의 데모 킷에서 녹화 & 저장 할 수 있도록 하자. <p>
비디오 재생을 멈추려면 video 화면에서 마우스를 클릭 한 이후 **ii** 라고 키보드 입력을 하면 강제종료 된다. 

In [10]:
show_video('SF', 'RES/smart_factory_sample.mp4')

여러분들이 저장할 비디오의 총 녹화 길이와 저장될 파일 이름을 아래와 같이 선언한다.

In [15]:
##################################################
# Video recording time (seconds)
REC_TIME_SEC = 120
# Video file name
REC_FILE_NAME = 'factory.mp4'
##################################################

아래 코드는 OpenCV를 사용해 camera 로 부터 frame data 를 받고 이를 비디오 파일로 저장하는 기능을 한다.

In [8]:
# OpenCV 를 사용하여 camera steam 을 연다.
cap = cv2.VideoCapture(1)
W = int(cap.get(3))
H = int(cap.get(4))
FPS = cap.get(5)
FRAME_CNT = 0

# 저장할 비디오 파일을 선언한다.
video_out = cv2.VideoWriter('SAVE/' + REC_FILE_NAME, cv2.VideoWriter_fourcc(*'DIVX'), FPS, (W, H))

In [9]:
# Set up widgets for a Jupyter Notebook environment
widget = widgets.Image(
    layout = widgets.Layout(border="solid")
)
display.display(widget)

while (cap.isOpened()):
    try:
        # Capture frame by frame
        ret, frame = cap.read()

        # Save
        video_out.write(frame)
        
        # Print
        jpg_stream = cv2.imencode(".jpg", frame)[1].tobytes()
        widget.value = jpg_stream

        FRAME_CNT = FRAME_CNT + 1
        if FRAME_CNT >= FPS * REC_TIME_SEC:
            print('Stop recording for {} seconds'.format(REC_TIME_SEC))
            break

        if (FRAME_CNT % FPS) == 0:
            print('Recording.. {:.2f}%'.format((100 * FRAME_CNT) / (REC_TIME_SEC * FPS)))

    except KeyboardInterrupt:  # input the 'ii' key
        break

video_out.release()
cap.release()
cv2.destroyAllWindows()

Image(value=b'', layout="Layout(border='solid')")

Recording.. 10.00%
Recording.. 20.00%
Recording.. 30.00%
Recording.. 40.00%
Recording.. 50.00%
Recording.. 60.00%
Recording.. 70.00%
Recording.. 80.00%
Recording.. 90.00%
Stop recording for 10 seconds


## **5. 비디오 파일이 잘 생성됐는지 확인하기**

In [29]:
# Replay the saved video file
show_video('','SAVE/' + REC_FILE_NAME)

Image(value=b'', layout="Layout(border='solid')")

C:\Users\Heedong\AppData\Local\Temp\ipykernel_27788\1550592973.py:25: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  jpg_stream = cv2.imencode(".jpg", img)[1].tostring()
